<a href="https://colab.research.google.com/github/junjibea/2021-2-Interpark-review/blob/main/Review_Crawling_Sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 라이브러리 import
import pandas as pd
import numpy as np

from selenium import webdriver
from tqdm.notebook import tqdm
import time

처음 팝업창부터 제거

In [ ]:
def remove_pop():
    button = driver.find_elements_by_tag_name("button")
    button[1].click()

# 공연정보 추출

In [ ]:
def get_musical_title(): 
    title = driver.find_element_by_css_selector(".prdTitle")
    print(title.text)
    
    return title.text

#### 공연 장소 기간

In [ ]:
def get_place_time():
    place_time = driver.find_elements_by_css_selector(".infoItem")[:2]
    
    return place_time[0].text[:-5], place_time[1].text[5:]

#### 배우 정보 더보기

In [ ]:
def open_actor():
    button_2 = driver.find_element_by_css_selector(".contentToggleBtn")
    button_2.click()

In [ ]:
def get_actor():
    actor_list = []
    actors = driver.find_elements_by_css_selector(".castingList .castingName")
    for a in actors:
        actor_list.append(a.text)
        
    return actor_list

# 공연후기 추출

#### 공연후기 탭 이동

In [ ]:
def open_review():
    a = driver.find_elements_by_css_selector(".navList .navItem")
    a[3].click()

#### review title

In [ ]:
def get_review_title():
    r_t_list = []
    r_t = driver.find_elements_by_css_selector(".bbsTitle")[1:]
    
    for t in r_t:
        r_t_list.append(t.text)
    
    return r_t_list

#### review text

In [ ]:
def get_review_text():
    r_txt_list = []
    r_txt = driver.find_elements_by_css_selector(".bbsText")[1:]

    for txt in r_txt:
        r_txt_list.append(txt.text)
    
    return r_txt_list

#### review rate

In [ ]:
def get_review_rate():
    rate_list = []
    rates = driver.find_elements_by_css_selector(".prdStarIcon")[3:]
    for rate in rates:
        rate_num = int(rate.get_attribute("data-star"))
        rate_list.append(rate_num)
        
    return rate_list

#### review info

In [ ]:
def get_review_info():
    e = driver.find_elements_by_css_selector(".bbsItemInfo .bbsItemInfoList")
    
    id_list, date_list, like_list = [], [], []
    
    ids = e[0 + 4:64:4]
    for i in ids:
        id_list.append(i.text[:-3])
    

    date = e[1 + 4:64:4]
    for i in date:
        date_list.append(i.text)

    like = e[3 + 4:64:4]
    for i in like:
        like_list.append(i.text[3:][:-5])
        
    return id_list, date_list, like_list

#### move page

In [ ]:
def move_to_page(i):
    if i != 0:
        p = driver.find_elements_by_css_selector(".pageNumWrap .pageNumBtn")
        p[i].click()

In [ ]:
url_list = ["https://tickets.interpark.com/goods/21005689", "https://tickets.interpark.com/goods/21005816"]

In [ ]:
url_list = ["https://tickets.interpark.com/goods/21005689", "https://tickets.interpark.com/goods/21005816"]
musicals_reviews_df = pd.DataFrame()
musicals_info = []

driver = webdriver.Chrome("C:/Users/82102/Jupyter Notebook/chromedriver_win32/chromedriver") # 본인이 갖고 있는 driver 경로

for k in range(2):
    # url 선택
    url = url_list[k]
    driver.get(url)
    
    # 팝업 창 제거
    if k == 0:
        remove_pop()
    # 공연정보 
    musical_info = []
    
    musical_info.append(get_musical_title())

    place_time = get_place_time()
    musical_info.append(place_time[0])
    musical_info.append(place_time[1])

    open_actor()
    musical_info.append(get_actor())
    
    musicals_info.append(musical_info)
    
    # Review
    open_review()

    df = pd.DataFrame()
    review_title, review_text, ids, date, like, rate = [], [], [], [], [], []
    for i in tqdm(range(0, 3)): # page 1, 2, 3
        move_to_page(i)

        time.sleep(0.5)

        review_info = get_review_info()
        ids.append(review_info[0])
        date.append(review_info[1])
        like.append(review_info[2])

        review_title.append(get_review_title())
        review_text.append(get_review_text())

        rate.append(get_review_rate())

    df["title"] = [y for x in review_title for y in x]
    df["text"] = [y for x in review_text for y in x]
    df["id"] = [y for x in ids for y in x]
    df["date"] = [y for x in date for y in x]
    df["like"] = [y for x in like for y in x]
    df["rate"] = [y for x in rate for y in x]
    # 노최유하 추가
    df["show"] = [musical_info[0] for i in range(len(df))]
    
    musicals_reviews_df = pd.concat([musicals_reviews_df, df], axis = 0)

    
driver.close()

musicals_info_df = pd.DataFrame(musicals_info)

musicals_reviews_df.reset_index(inplace = True)

musicals_reviews_df

In [ ]:
musicals_info_df.to_csv("musicals_info.csv", encoding = "cp949", index = None)
musicals_reviews_df.to_csv("musicals_review.csv", encoding = "cp949", index = None)

---

# 부록

아이디에서 예매자 텍스트 제거